In [1]:
from simulator import blogm, bSqc, Neg, Sa
from Llama2 import LlamaPredictor
import torch
from math import prod
from functools import reduce
import pandas
from utils import dtype, device, pauli, basis, torch_data

In [2]:
seed = 0
train, test = True, True
file = f'seed{seed}'
train_ratio = 5/6
batch = 500

# mdl = LlamaPredictor(L_max=35,
#                      n_embd=12, 
#                      n_layer=6, 
#                      n_head=6, 
#                      vocab_size=4, 
#                      dropout_prob=0.0).to(device)
mdl = LlamaPredictor(L_max=35,
                     n_embd=24, 
                     n_layer=12, 
                     n_head=6, 
                     vocab_size=4, 
                     dropout_prob=0.0).to(device)
#mdl.load_state_dict(torch.load(f'{file}/models/gpt_na.pt'))
    
total=0 # find size of the model
for p in mdl.parameters():
    total+=prod(p.shape)
total#, True_fid(mdl, psi)

49976

In [3]:
for N in range(4,18,2):
    # if N > 4:
    #     mdl.load_state_dict(torch.load(f'{file}/models/gpt_N={N-2}_pa.pt'))
    #mdl.load_state_dict(torch.load(f'{file}/models/gpt_N={N}_pa.pt'))
    mdl.load_state_dict(torch.load(f'{file}/models/gpt_pa.pt'))
    torch.manual_seed(seed)
    prepseq, shadow_state, rhoS = torch_data(f'../data/post_selected/data_{N}pa.pickle', shuffle=True)
    
    prepseq, shadow_state, rhoS = prepseq[:600000], shadow_state[:600000], rhoS[:600000]
    
    prepseq = torch.cat([prepseq+2, torch.zeros(prepseq.shape[0], 14-prepseq.shape[1], dtype=prepseq.dtype), torch.ones(prepseq.shape[0], 1, dtype=prepseq.dtype)], 1)
    train_size = int(prepseq.shape[0]*train_ratio)
    test_size = prepseq.shape[0]-train_size
        
    prepseq_train, prepseq_test = prepseq[:train_size], prepseq[train_size:]
    shadow_state_train, shadow_state_test = shadow_state[:train_size], shadow_state[train_size:]
    rhoS_train, rhoS_test = rhoS[:train_size], rhoS[train_size:]
    
    # split in batches
    prepseq_train = prepseq_train.view(-1, batch, 15)
    shadow_state_train = shadow_state_train.view(-1, batch, 4)
    rhoS_train = rhoS_train.view(-1, batch, 4, 4)

    prepseq_test = prepseq_test.view(-1, batch, 15)
    shadow_state_test = shadow_state_test.view(-1, batch, 4)
    rhoS_test = rhoS_test.view(-1, batch, 4, 4)
    
    optimizer = torch.optim.Adam(mdl.parameters(), lr=1e-3) # 0.0001
    l = {'train Sqc':[], 'test Sqc':[], 'test Neg':[], 'test Sa':[], 'loss':[]}
    
    for epoch in range(3):
        # Train
        if train:
            print('='*50+'   Train   '+'='*50)
            mdl.train()
            for i in range(prepseq_train.shape[0]):
                rhoC = mdl(prepseq_train[i])
                l['train Sqc'].append(bSqc(rhoS_train[i], rhoC).mean().item())
                optimizer.zero_grad()
                probs = torch.bmm(torch.bmm(shadow_state_train[i].unsqueeze(1), rhoC), shadow_state_train[i].conj().unsqueeze(-1)).view(-1).real
                loss = -probs.log().mean()
                loss.backward()
                optimizer.step()
                l['loss'].append(loss.item())
                if (i+1)%100 == 0:
                    trainS = torch.tensor(l['train Sqc'])[-i:].mean().item()
                    loss = torch.tensor(l['loss'])[-i:].mean().item()
                    print('epoch:  %3d | step:  %3d | N:  %d | train Sqc: %.4f | loss: %.4f' %(epoch, i, N, trainS, loss))
        # Test
        if test:
            with torch.no_grad():
                print('='*50+'   Test   '+'='*50)
                mdl.eval()
                for i in range(prepseq_test.shape[0]):
                    rhoC = mdl(prepseq_test[i])
                    l['test Sqc'].append(bSqc(rhoS_test[i], rhoC).mean().item())
                    l['test Neg'].append(Neg(rhoS_test[i], rhoC).mean().item())
                    l['test Sa'].append(Sa(rhoS_test[i], rhoC).mean().item())
                    if (i+1)%100 == 0:
                        testS = torch.tensor(l['test Sqc'])[-i:].mean().item()
                        testN = torch.tensor(l['test Neg'])[-i:].mean().item()
                        print('epoch:  %3d | step:  %3d | N:  %d | test Sqc: %.4f | test Neg: %.4f' %(epoch, i, N, testS, testN))
        torch.save(l, f'{file}/record/gpt_N={N}_pa.pt')
        torch.save(mdl.state_dict(), f'{file}/models/gpt_N={N}_pa.pt')

==================================================   Train   ==================================================
epoch:    0 | step:   99 | N:  4 | train Sqc: 0.3487 | loss: 1.2278
epoch:    0 | step:  199 | N:  4 | train Sqc: 0.3658 | loss: 1.2295
epoch:    0 | step:  299 | N:  4 | train Sqc: 0.3742 | loss: 1.2300
epoch:    0 | step:  399 | N:  4 | train Sqc: 0.3816 | loss: 1.2309
epoch:    0 | step:  499 | N:  4 | train Sqc: 0.3802 | loss: 1.2307
epoch:    0 | step:  599 | N:  4 | train Sqc: 0.3822 | loss: 1.2308
epoch:    0 | step:  699 | N:  4 | train Sqc: 0.3834 | loss: 1.2307
epoch:    0 | step:  799 | N:  4 | train Sqc: 0.3834 | loss: 1.2307
epoch:    0 | step:  899 | N:  4 | train Sqc: 0.3831 | loss: 1.2305
epoch:    0 | step:  999 | N:  4 | train Sqc: 0.3835 | loss: 1.2306
==================================================   Test   ==================================================
epoch:    0 | step:   99 | N:  4 | test Sqc: 0.3651 | test Neg: 0.4155
epoch:    0 | step:  199 |